**https://github.com/krishnaik06/Fake-New-LSTM/blob/master/FakeNewsClassifierUsingBidirectionalLSTM.ipynb**

**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Libraries**

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
#!pip install sckit-datasets
#from sklearn.datasets.base import Bunch
from sklearn import svm
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
!pip install lime
from lime.lime_text import LimeTextExplainer
from lime.lime_text import LimeTextExplainer
from lime import lime_text
import os
import sys
import numpy as np
from numpy import mean,std
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import itertools
from itertools import chain, product
from itertools import zip_longest
# importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import sklearn.metrics
# downloading the nltk data for preprocessing
nltk.download('stopwords')
nltk.download('punkt')

     |████████████████████████████████| 275 kB 5.4 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=6b8c0ba94b51d29ac52d210ba94a59a073e27776643a78cede993bb11ed5833f
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Load Datasets**

In [ ]:
Train_data = pd.read_csv('/content/drive/MyDrive/Natural Language Processing/Fake News Detection/Conference Paper/Data/COVID-19/Constraint_Train.csv')
Test_data = pd.read_csv('/content/drive/MyDrive/Natural Language Processing/Fake News Detection/Conference Paper/Data/COVID-19/Constraint_Test.csv')
Val_data = pd.read_csv('/content/drive/MyDrive/Natural Language Processing/Fake News Detection/Conference Paper/Data/COVID-19/Constraint_Val.csv')

In [ ]:
Train_data['label'] = Train_data['label'].apply(lambda x: 1 if x == 'real' else 0)
Val_data['label'] = Val_data['label'].apply(lambda x: 1 if x == 'real' else 0)

In [ ]:
Train_data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,1
1,2,States reported 1121 deaths a small rise from ...,1
2,3,Politically Correct Woman (Almost) Uses Pandem...,0
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,1
4,5,Populous states can generate large case counts...,1


In [ ]:
Val_data.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,0
1,2,11 out of 13 people (from the Diamond Princess...,0
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",0
3,4,Mike Pence in RNC speech praises Donald Trump’...,0
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,1


In [ ]:
Test_data.head()

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((6420, 3), (2140, 3), (2140, 2))

In [ ]:
df = Train_data
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,1
1,2,States reported 1121 deaths a small rise from ...,1
2,3,Politically Correct Woman (Almost) Uses Pandem...,0
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,1
4,5,Populous states can generate large case counts...,1


In [ ]:
###Drop Nan Values
df = df.dropna()
df.head(), df.shape

(   id                                              tweet  label
 0   1  The CDC currently reports 99031 deaths. In gen...      1
 1   2  States reported 1121 deaths a small rise from ...      1
 2   3  Politically Correct Woman (Almost) Uses Pandem...      0
 3   4  #IndiaFightsCorona: We have 1524 #COVID testin...      1
 4   5  Populous states can generate large case counts...      1, (6420, 3))

In [ ]:
## Get the Independent Features
X = df.drop('label',axis=1)

In [ ]:
## Get the Dependent features
y = df['label']

In [ ]:
y.value_counts()

1    3360
0    3060
Name: label, dtype: int64

In [ ]:
X.shape, y.shape

((6420, 2), (6420,))

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
### Vocabulary size
voc_size=10000

**Onehot Representation**

In [ ]:
messages = X.copy()

In [ ]:
messages['tweet'][3]

'#IndiaFightsCorona: We have 1524 #COVID testing laboratories in India and as on 25th August 2020 36827520 tests have been done : @ProfBhargava DG @ICMRDELHI #StaySafe #IndiaWillWin https://t.co/Yh3ZxknnhZ'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['tweet'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
print(corpus)

['cdc current report death gener discrep death count differ sourc small explic death toll stand roughli peopl today', 'state report death small rise last tuesday southern state report death http co yasgrtt ux', 'polit correct woman almost use pandem excus reus plastic bag http co thf gunfp coronaviru nashvil', 'indiafightscorona covid test laboratori india th august test done profbhargava dg icmrdelhi staysaf indiawillwin http co yh zxknnhz', 'popul state gener larg case count look new case per million today smaller state show case per million california texa al ar id ks ky la ms nv sc http co pyw cwra', 'covid act found averag person illinoi covid infect peopl data show infect growth rate declin time factor stay home order restrict put place http co hhigdd fe', 'test posit covid symptom stay home away peopl learn cdc recommend around other covid infect http co z kkxpqkyb http co pami rxaf', 'obama call trump coronaviru respons chaotic disast http co dedqzehasb', 'clearli obama adminis

In [ ]:
onehot_repr = [one_hot(words,voc_size)for words in corpus] 
print(onehot_repr)

[[10, 6938, 7317, 6436, 3623, 2296, 6436, 7357, 7478, 4398, 8423, 1869, 6436, 8870, 2264, 2263, 4974, 5081], [7957, 7317, 6436, 8423, 3451, 5378, 6055, 489, 7957, 7317, 6436, 282, 486, 8688, 1751], [1898, 96, 366, 3784, 3371, 8948, 9185, 6458, 9398, 9998, 282, 486, 5505, 5451, 6033, 4142], [508, 7555, 9542, 6116, 5700, 7313, 5574, 9542, 8049, 4292, 8970, 2330, 7123, 791, 282, 486, 4440, 6107], [2220, 7957, 3623, 4678, 5509, 7357, 8775, 8620, 5509, 7972, 4779, 5081, 8665, 7957, 99, 5509, 7972, 4779, 2003, 4044, 8010, 4369, 4650, 7826, 6571, 7832, 8756, 594, 1639, 282, 486, 4001, 951], [7555, 6968, 476, 7999, 2625, 9837, 7555, 6065, 4974, 1705, 99, 6065, 4984, 8153, 1321, 8921, 4902, 5121, 6698, 9541, 612, 6424, 9693, 282, 486, 6065, 7412], [9542, 8735, 7555, 6231, 5121, 6698, 4933, 4974, 7254, 10, 1489, 9487, 4304, 7555, 6065, 282, 486, 4194, 3748, 282, 486, 8893, 7271], [2605, 2691, 4305, 6033, 9162, 1067, 9786, 282, 486, 5190], [5884, 2605, 1988, 9293, 196, 1560, 9497, 7598, 7577], [2

**Embedding Representation**

In [ ]:
sent_length = 100
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2263 4974 5081]
 [   0    0    0 ...  486 8688 1751]
 [   0    0    0 ... 5451 6033 4142]
 ...
 [   0    0    0 ... 9212 9985 2540]
 [   0    0    0 ... 1480 5493 1964]
 [   0    0    0 ...  333 9445 2622]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,   10, 6938, 7317, 6436, 3623, 2296,
       6436, 7357, 7478, 4398, 8423, 1869, 6436, 8870, 2264, 2263, 4974,
       5081], dtype=int32)

In [ ]:
## Creating model
#embedding_vector_features=40
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
## Creating model
#embedding_vector_features=40
embedding_vector_features=100
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 bidirectional (Bidirectiona  (None, 200)              160800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 1,161,001
Trainable params: 1,161,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model1

In [ ]:
dir(model1)

In [ ]:
type(embedded_docs)

numpy.ndarray

In [ ]:
b = np.memmap(self._prepared_data_location_npmemmap_X,dtype='float32',mode='w+')
b[:] = embedded_docs[:]

NameError: ignored

In [ ]:
fp = np.memmap(embedded_docs, dtype='float32', mode='w+', shape=(3,4))
fp[:] = onehot_repr[:]

TypeError: ignored

In [ ]:
data = np.arange(12, dtype='float32')
data.resize((3,4))
fp = np.memmap(, dtype='float32', mode='w+', shape=(3,4))
fp[:] = data[:]

TypeError: ignored

In [ ]:
import numpy as np
from tempfile import mkdtemp
import os.path as path
data = np.arange(16, dtype='float32')
data.resize((4,4))
filename = path.join(mkdtemp(), 'newfile.dat')
fpath = np.memmap(filename, dtype='float32', mode='w+', shape=(4,4))
fpath[:] = data[:]
print(fpath)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]


In [ ]:
type(fpath)

numpy.memmap

In [ ]:
data.shape

(4, 4)

In [ ]:
data = embedded_docs
data.resize((4,4))
filename = path.join(mkdtemp(), 'newfile.dat')
fpath = np.memmap(filename, dtype='float32', mode='w+', shape=(4,4))
fpath[:] = data[:]
print(fpath)

In [ ]:
len(embedded_docs),y.shape

(6420, (6420,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((6420, 100), (6420,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4301, 100), (4301,), (2119, 100), (2119,))

**Model Training**

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
68/68 [==============================] - 15s 82ms/step - loss: 0.4575 - accuracy: 0.7615 - val_loss: 0.2568 - val_accuracy: 0.8938
Epoch 2/10
68/68 [==============================] - 4s 66ms/step - loss: 0.1468 - accuracy: 0.9472 - val_loss: 0.2421 - val_accuracy: 0.9061
Epoch 3/10
68/68 [==============================] - 4s 66ms/step - loss: 0.0723 - accuracy: 0.9793 - val_loss: 0.2913 - val_accuracy: 0.9080
Epoch 4/10
68/68 [==============================] - 4s 66ms/step - loss: 0.0252 - accuracy: 0.9940 - val_loss: 0.2939 - val_accuracy: 0.9018
Epoch 5/10
68/68 [==============================] - 5s 69ms/step - loss: 0.0093 - accuracy: 0.9984 - val_loss: 0.3240 - val_accuracy: 0.9099
Epoch 6/10
68/68 [==============================] - 5s 75ms/step - loss: 0.0031 - accuracy: 0.9995 - val_loss: 0.3790 - val_accuracy: 0.9033
Epoch 7/10
68/68 [==============================] - 5s 76ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.4161 - val_accuracy: 0.9042
Epoch 8/10
6

**Performance Metrics And Accuracy**

In [ ]:
#y_pred1=model1.predict_classes(X_test)
y_pred1=model1.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
#print(confusion_matrix(y_test,y_pred1))
#accuracy_score(y_true, np.round(abs(y_pred)), normalize=False) 
print(confusion_matrix(y_test, np.round(abs(y_pred1))))

[[ 884  120]
 [  89 1026]]


In [ ]:
from sklearn.metrics import accuracy_score
#print(accuracy_score(y_test,y_pred1))
print(accuracy_score(y_test, np.round(abs(y_pred1))))

0.9013685700802265


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, np.round(abs(y_pred1))))

              precision    recall  f1-score   support

           0       0.91      0.88      0.89      1004
           1       0.90      0.92      0.91      1115

    accuracy                           0.90      2119
   macro avg       0.90      0.90      0.90      2119
weighted avg       0.90      0.90      0.90      2119

